In [1]:
!pip -q install transformers
!pip -q install peft
!pip -q install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.7 MB/s eta 0:00:00


In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import default_data_collator, get_linear_schedule_with_warmup
from datasets import load_dataset, Dataset

from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType
from peft import PeftModel, PeftConfig

import torch
from torch.utils.data import DataLoader

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from tqdm import tqdm
from datetime import datetime
import numpy as np

import re

# mount google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [30]:
def check_str(x):
  x = re.findall('[\u3002\uff1b\uff0c\uff1a\u201c\u201d\uff08\uff09\u3001\uff1f\u300a\u300b\u4e00-\u9fa5\w]',x)
  x = ''.join(x)
  #x = x.replace('\u3000', '')
  #x = x.replace('\xa0', '')

  return {'sentence': x + '\n'}

In [33]:
with open('/content/gdrive/MyDrive/chinese_dataset/2023-06-29 天龍八部.txt') as f:
  data = f.readlines()[40:]

data = list(filter(lambda x: len(x.strip()) >= 30, data))
data = list(map(check_str, data))
print('length:', len(data))

length: 11022


In [34]:
nn = 0
for row in data:
  print(row, len(row['sentence']))
  nn += 1
  if nn >= 10:
    break

{'sentence': '天龍八部這八種神道精怪，各有奇特個性和神通，雖是人間之外的眾生，卻也有塵世的歡喜和悲苦。這部小說裡沒有神道精怪，只是借用這個佛經名詞，以象徵一些現世人物，就像水滸中有母夜叉孫二孃、摩雲金翅歐鵬。\n'} 98
{'sentence': '青光閃動，一柄青鋼劍倏地刺出，指向中年漢子左肩，使劍少年不等劍招用老，腕抖劍斜，劍鋒已削向那漢子右頸。那中年漢子豎劍擋格，錚的一聲響，雙劍相擊，嗡嗡作聲，震聲未絕，雙劍劍光霍霍，已拆了三招。中年漢子長劍猛地擊落，直砍少年頂門。那少年避向右側，左手劍訣一引，青鋼劍疾刺那漢子大腿。\n'} 140
{'sentence': '練武廳東邊坐著二人。上首是個四十左右的中年道姑，鐵青著臉，嘴脣緊閉。下首是個五十餘歲的老者，右手捻著長鬚，神情甚是得意。兩人的座位相距一丈有餘，身後各站著二十餘名男女弟子。西邊一排椅子上坐著十餘位賓客。東西雙方的目光都集註於場中二人的角鬥。\n'} 121
{'sentence': '眼見那少年與中年漢子已拆到七十餘招，劍招越來越緊，兀自未分勝敗。突然中年漢子一劍揮出，用力猛了，身子微微一晃，似欲摔跌。西邊賓客中一個身穿青衫的年輕男子忍不住“嗤”的一聲笑。他隨即知道失態，忙伸手按住了口。\n'} 104
{'sentence': '便在這時，場中少年左手呼的一掌拍出，擊向那漢子後心。那漢子向前跨出一步避開，手中長劍驀地圈轉，喝一聲：“著”那少年左腿已然中劍，腿下一個踉蹌，長劍在地下一撐，站直身子待欲再鬥，那中年漢子已還劍入鞘，笑道：“褚師弟，承讓，承讓，傷得不厲害麼”那少年臉色蒼白，咬著嘴脣道：“多謝龔師兄劍下留情。”\n'} 147
{'sentence': '那長鬚老者滿臉得色，微微一笑，說道：“東宗已勝了三陣，看來這劍湖宮又要讓東宗再住五年了。辛師妹，咱們還須比下去麼”坐在他上首的那中年道姑強忍怒氣，說道：“左師兄果然調教得好徒兒。但不知左師兄對無量玉壁的鑽研，這五年來可已大有心得麼”長鬚老者向她瞪了一眼，正色道：“師妹怎地忘了本派的規矩”那道姑哼了一聲，便不再說下去了。\n'} 161
{'sentence': '這老者姓左，名叫子穆，是“無量劍”東宗的掌門。那道姑姓辛，道號雙清，是“無量劍”西宗掌門。\n'} 46
{'sentence': '“無量劍”原分東、北、西三宗，北宗

In [38]:
device = "cuda"
model_name_or_path = "bigscience/mt0-small"
tokenizer_name_or_path = "bigscience/mt0-small"

#checkpoint_name = "financial_sentiment_analysis_lora_v1.pt"
text_column = "sentence"
label_column = "text_label"
max_length = 512
lr = 1e-3
num_epochs = 5
batch_size = 8

In [37]:
continue_train = False
# creating model
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

peft_model_id = f"/content/gdrive/MyDrive/chinese_dataset/bigscience/mt0-small_LORA_202307020215"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
if continue_train:
  print('Continue to train...')
  model = PeftModel.from_pretrained(model, peft_model_id)
  for name, param in model.named_parameters():
    if 'lora' in name or 'Lora' in name:
        param.requires_grad = True
else:
  model = get_peft_model(model, peft_config)

model.print_trainable_parameters()

#model

Some weights of MT5ForConditionalGeneration were not initialized from the model checkpoint at bigscience/mt0-small and are newly initialized: ['decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 344064 || all params: 300520832 || trainable%: 0.11448923447676333


In [43]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

def preprocess_fn(sample):
    sentence = sample['sentence']
    idx = np.random.randint(5, 15, 1)[0]
    inputs = sentence[:idx].rstrip()
    targets = sentence[idx:].rstrip()

    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")

    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels

    return model_inputs

In [ ]:
#tokenizer('哈囉', max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")['input_ids'].shape

In [44]:
dataset = Dataset.from_list(data[:8000])
#dataset = dataset.shuffle(seed=42)
train_valid = dataset.train_test_split(test_size=0.2)
train_dataset = train_valid['train']
valid_dataset = train_valid['test']


{'sentence': '喬峰藝成以來，雖然身經百戰，從未一敗，但同時與這許多高手對敵，卻也是生平未遇之險。這時他酒意已有十分，內力鼓盪，酒意更漸漸湧將上來，雙掌飛舞，逼得眾高手無法近身。\n'}


In [ ]:
processed_train = train_dataset.map(
    preprocess_fn,
    batched=False,
    num_proc=1,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

processed_valid = valid_dataset.map(
    preprocess_fn,
    batched=False,
    num_proc=1,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

train_dataloader = DataLoader(
    processed_train, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)

valid_dataloader = DataLoader(
    processed_valid, shuffle=False, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)

print('train nums', len(train_dataset), 'valid nums', len(valid_dataset))

Running tokenizer on dataset:   0%|          | 0/6400 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/1600 [00:00<?, ? examples/s]

train nums 6400 valid nums 1600


In [ ]:
## check input data
for i in train_dataloader:
  ins = i['input_ids'].detach().cpu().numpy()[0, 0, :]
  las = i['labels'].detach().cpu().numpy()[0, 0, :]
  las[las == -100] = 0
  print(ins.shape)
  print(las.shape)

  print(tokenizer.decode(ins, skip_special_tokens=True))
  print(tokenizer.decode(las, skip_special_tokens=True))
  break

(128,)
(128,)
蘇星河道:
“你為什麼不許我自盡”虛竹道:“出家人慈悲為本,我自然不忍見你喪命。”蘇星河道:“你放開我,我是決計不想活了。”虛竹道:“我不放。”蘇星河道:“難道你一輩子捉住我不放”虛竹心想這個話倒也不錯,便將他身子倒了轉來,頭上腳下的放好,說道:“好,放便放你,卻不許你自盡。”


In [ ]:
# optimizer and lr scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
# training and evaluation
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: torch.squeeze(v).to(device) for k, v in batch.items()}
        outputs = model(**batch)
        #print(outputs.keys())
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    # saving checkpoints
    time_now = datetime.now().strftime('%Y%m%d%H%M')
    #peft_model_id = f"/content/gdrive/MyDrive/chinese_dataset/{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
    peft_model_id = f"/content/gdrive/MyDrive/chinese_dataset/{model_name_or_path}_{peft_config.peft_type}_{time_now}"
    model.save_pretrained(peft_model_id)

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(valid_dataloader)):
        batch = {k: torch.squeeze(v).to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(valid_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print()
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([3, 128])


In [ ]:
!du -h "/content/gdrive/MyDrive/chinese_dataset/bigscience/mt0-small_LORA_202307020215/"

1.4M	/content/gdrive/MyDrive/chinese_dataset/bigscience/mt0-small_LORA_SEQ_2_SEQ_LM/


In [ ]:
from peft import PeftModel, PeftConfig

peft_model_id = f"/content/gdrive/MyDrive/chinese_dataset/bigscience/mt0-small_LORA_202307020324"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

Some weights of MT5ForConditionalGeneration were not initialized from the model checkpoint at bigscience/mt0-small and are newly initialized: ['decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
## original pre-trained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

Some weights of MT5ForConditionalGeneration were not initialized from the model checkpoint at bigscience/mt0-small and are newly initialized: ['decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
del model
model

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [ ]:
model.eval()

i = 9
input_text = processed_valid[i]['sentence']
label_ids = processed_valid[i]['labels'][0]
label_ids = np.array(label_ids)
label_ids[label_ids == -100] = 0
#print(label_ids)

#input_text = '12星座分別有哪幾種?'
inputs = tokenizer(input_text, return_tensors="pt")
label_texts = tokenizer.decode(label_ids)
#print(input_text)
#print(processed_valid[i]['input_ids'])
#print()
#print(label_texts)

orig_inputs = input_text
all_texts = ''

for j in tqdm(range(10)):
  with torch.no_grad():
    outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=120, temperature=0.8,
                               num_beams=1, repetition_penalty=1.3, do_sample=True)

    output_text = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)

  all_texts += output_text[0] + '\n'

  #print(all_texts)
  input_text = all_texts
  inputs = tokenizer(input_text, return_tensors="pt")

#print(orig_inputs)
print()
print('Input:', orig_inputs)
print(all_texts)


100%|██████████| 10/10 [00:37<00:00,  3.75s/it]


Input: 車行轔轔，日夜不停。玄難、鄧百川、康廣陵等均是當世武林大豪，這時武功全失，成為隨人擺佈的囚徒。眾人只約莫感到，一行人是向東南方行。
四長,道:“你和他比此人要不退位。”這時也叫一聲:「我好下,沒聽到自己”
,便道:“是我再要給我幫你自己。”說著兩人。另一來三長兄說了話。
,我不得跟你從心過來。真想出人自己也沒了這個事了。”
我說他對你的話,你沒得到這句中。”
acá,你那位不算之處。這番詩是個無名同理。我正打他們的子弟和他相隔一小老兵、誰也說話了,只不過她們將何來用到這個人給你幫自己好”
,問你說了一個”
:“我那麼人 종류。你的這種子說好,當然得出自來。”
,我說你什麼事”她回來。
。你沒人太不順重,再來一張詩的招呼馬上,想是誰之事ແລະ應有幫我們。”
這番詩來自那人,我只怕為什麼要讓你幫得。

